In [2]:
import tensorflow as tf 

x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5], [1, 7, 5, 5], 
                                                        [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
   sess.run(tf.global_variables_initializer())

   for step in range(2001):
       sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
       if step % 200 == 0:
           print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
0 1.5039003
200 0.6819203
400 0.57709754
600 0.4848227
800 0.3949597
1000 0.3053977
1200 0.2393079
1400 0.2170772
1600 0.19850305
1800 0.18271312
2000 0.16914153


In [20]:
# Testing & One-hot encoding
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.math.argmax(a, 1)))
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], 
                                          [1, 3, 4, 3], 
                                          [1, 1, 0, 1]]})
    print(all, sess.run(tf.arg_max(all, 1)))



[[9.9999976e-01 9.3369987e-09 2.6151835e-07]] [0]
[[9.9999976e-01 9.3369987e-09 2.6151835e-07]
 [9.4655734e-01 2.9309599e-03 5.0511684e-02]
 [5.6236732e-01 3.0276141e-01 1.3487126e-01]] [0 0 0]


In [22]:
import numpy as np
# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Launch graph
with tf.Session() as sess:
   sess.run(tf.global_variables_initializer())

   for step in range(2000):
       sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
       if step % 100 == 0:
           loss, acc = sess.run([cost, accuracy], feed_dict={
                                X: x_data, Y: y_data})
           print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
               step, loss, acc))

   # Let's see if we can predict
   pred = sess.run(prediction, feed_dict={X: x_data})
   # y_data: (N,1) = flatten => (N, ) matches pred.shape
   for p, y in zip(pred, y_data.flatten()):
       print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


OSError: data-04-zoo.csv not found.